### here we will standardize the data for better merge performance
process stages:
1. change country names in results table to fit noaa_countries data
2. add noaa_country_code from locations to results by city
3. change city name in results to fit city name in locations
4. merge location_id from locations to results table by city and noaa_country_code code

In [1]:
import requests
import sqlite3
import pandas as pd

# NOAA API token
api_token = 'rDZYpxCxcNrIUzeserZMcYDaaHCNPcFZ'

# connect to the DB
conn = sqlite3.connect('C:/Users/Mark Rozenberg/Foobal-Climate/Data/Main_DB.db')

#### stage 1

In [3]:
pd.read_sql(''' select * from results limit 3''', conn)

,index,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,noaa_country_code,location_id,mindate,maxdate,date_ts,mindate_ts,maxdate_ts,valid,location_date_id
0,0,1872-11-30 00:00:00,Scotland,England,0,0,Friendly,Glasgow,United Kingdom,0,UK,CITY:UK000006,1914-01-01,2024-08-31,1872-11-30,1914-01-01,2024-08-31,0,CITY:UK000006_1872-11-30
1,1,1873-03-08 00:00:00,England,Scotland,4,2,Friendly,London,United Kingdom,0,UK,CITY:UK000009,1904-02-01,2024-09-24,1873-03-08,1904-02-01,2024-09-24,0,CITY:UK000009_1873-03-08
2,2,1874-03-07 00:00:00,Scotland,England,2,1,Friendly,Glasgow,United Kingdom,0,UK,CITY:UK000006,1914-01-01,2024-08-31,1874-03-07,1914-01-01,2024-08-31,0,CITY:UK000006_1874-03-07


In [48]:
# Define the mapping dictionary
country_mapping = {
    'German DR': 'Germany',
    'Czechoslovakia': 'Czech Republic',
    'Yugoslavia': 'Serbia',
    'Serbia and Montenegro': 'Serbia',
    'Scotland' : 'United Kingdom',
    'England': 'United Kingdom',
    'Wales': 'United Kingdom',
    'China PR': 'China',
    'Northern Cyprus': 'Cyprus',
    'Northern Ireland': 'Ireland',
    'Republic of Ireland': 'Ireland',
    'Puerto Rico': 'Puerto Rico [United States]',
    'Gambia': 'Gambia, The',
    'South Korea': 'Korea, South',
    'North Korea': 'Korea, North',
    'Mali Federation': 'Mali',
    'Vietnam DR': 'Vietnam',
    'Vietnam Republic': 'Vietnam',
    'Ivory Coast': 'Cote D\'Ivoire',
    'Taiwan*': 'Taiwan',
    'North Macedonia': 'Macedonia',
    'Bahamas': 'Bahamas, The',
    'Micronesia': 'Federated States of Micronesia',
    'Samoa': 'American Samoa [United States]',
    'South Sudan': 'Sudan',
    'Congo': 'Congo (Brazzaville)',
    'DR Congo': 'Congo (Kinshasa)',
    'Belgian Congo': 'Congo (Kinshasa)',
    'Bermuda': 'Bermuda [United Kingdom]',
    'Guam': 'Guam [United States]',
    'Cayman Islands': 'Cayman Islands [United Kingdom]',
    'Gibraltar': 'Gibraltar [United Kingdom]',
    'United States Virgin Islands': 'Virgin Islands [United States]',
    'Cook Islands': 'Cook Islands [New Zealand]',
    'Northern Mariana Islands': 'Northern Mariana Islands [United States]' 
}

# Update the 'results' table in the SQLite database with the new city names
for old, new in country_mapping.items():
    conn.execute('''
    UPDATE results
    SET country = ?
    WHERE country = ?
    ''', (new, old))
conn.commit()

#### stage 2

In [51]:
conn.execute('ALTER TABLE results drop column noaa_country_code')
conn.execute('ALTER TABLE results ADD COLUMN noaa_country_code TEXT')
conn.execute('''
UPDATE results
SET noaa_country_code = (
    SELECT code
    FROM noaa_countries
    WHERE results.country = noaa_countries.name
)
''')
conn.commit()

In [5]:
# check the new column
susu = pd.read_sql('SELECT distinct country, noaa_country_code FROM results', conn)
susu.to_excel('C:/Users/Mark Rozenberg/Downloads/susu.xlsx', index=False)

#### stage 3

In [32]:
pd.read_sql('''
SELECT *
FROM results
limit 3
''', conn)

,index,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,noaa_country_code
0,0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,United Kingdom,0,UK
1,1,1873-03-08,England,Scotland,4,2,Friendly,London,United Kingdom,0,UK
2,2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,United Kingdom,0,UK


In [27]:
# Query to get all unique cities from the locations table that contain diacritics
pd.read_sql_query('''
SELECT DISTINCT city
FROM results
WHERE city GLOB '*[À-ÿ]*'
''', conn)

,city


In [26]:
import unicodedata
def remove_diacritics(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

# Fetch all cities from the results table
cities = pd.read_sql('SELECT DISTINCT city FROM results', conn)

# Update each city name by removing diacritics
for city in cities['city']:
    new_city = remove_diacritics(city).replace('ø', 'o').replace('ð', 'o')
    conn.execute('''
    UPDATE results
    SET city = ?
    WHERE city = ?
    ''', (new_city, city))

In [31]:
# test cities fit for merge. the output is only cities that are not in the locations table
wuwu = pd.read_sql_query('''
with part1 as (
select distinct city, noaa_country_code
from results
),
part2 as (
select distinct city, noaa_country_code
from locations
),
part3 as (
SELECT t1.city as results_city, t2.city as locations_city, t1.noaa_country_code as results_code, t2.noaa_country_code as locations_code
FROM part1 as t1
full join
part2 as t2
ON SUBSTR(t1.city, 1, 5) = SUBSTR(t2.city, 1, 5)
order by t1.city
)
select *
from part3
where (locations_city is null or results_city != locations_city)
    and results_code = locations_code
''', conn)
wuwu.to_excel('C:/Users/Mark Rozenberg/Downloads/check-2.xlsx', index=False)

In [30]:
# Define the mapping dictionary
city_mapping = {
    'Tel Aviv': 'Tel Aviv-Yafo',
    'Middlesbrough': 'Middlesborough',
    'Beer Sheva': 'Beersheba',
    'Bhubaneswar': 'Bhubaneshwar',
    'Chonburi': 'Chon Buri',
    'Dnipro': 'Dnipropetrovs\'k',
    'Donetsk': 'Donets\'k',
    'Frankfurt am Main': 'Frankfurt',
    'Frankfurt an der Oder': 'Frankfurt',
    'Johor Bahru': 'Johor Baharu',
    'Kazan': 'Kazan\'',
    'Kolkota': 'Kolkata',
    'Kota Bharu': 'Kota Baharu',
    'Luhansk': 'Luhans\'k',
    'Misrata': 'Misratah',
    'Palma de Mallorca': 'Palma De Mallorca',
    'Pointe-Noire': 'Pointe Noire',
    'Port-Gentil': 'Port Gentil',
    'Puerto la Cruz': 'Puerto La Cruz',
    'Samarqand': 'Samarkand',
    'Suphan Buri': 'Supham Buri',
    'Surabaya': 'Surabaja',
    'Usti nad Labem': 'Usti Nad Labem',
    'Vitsebsk': 'Vitsyebsk',
    'Washington, D.C.': 'Washington D.C.',
    'Wrocław': 'Wroclaw',
    'Zaporizhia': 'Zaporiyhzhya',
    'Belize City': 'Belize',
    'Guatemala City': 'Guatemala',
    'Kuwait City': 'Kuwait',
    'Panama City': 'Panama'
}

# Update the 'results' table in the SQLite database with the new city names
for old_city, new_city in city_mapping.items():
    conn.execute('''
    UPDATE results
    SET city = ?
    WHERE city = ?
    ''', (new_city, old_city))
conn.commit()

In [41]:
conn.execute('ALTER TABLE results ADD COLUMN location_id TEXT')
conn.execute('''
UPDATE results
SET location_id = (
    SELECT id
    FROM locations
    WHERE results.city = locations.city
    AND results.noaa_country_code = locations.noaa_country_code
)
''')
conn.commit()

In [11]:
conn.execute('ALTER TABLE results ADD COLUMN mindate TEXT')
conn.execute('ALTER TABLE results ADD COLUMN maxdate TEXT')
conn.execute('''
UPDATE results
SET mindate = (
    SELECT mindate
    FROM locations
    WHERE results.location_id = locations.id
),
maxdate = (
    SELECT maxdate
    FROM locations
    WHERE results.location_id = locations.id
)
''')
conn.commit()

In [12]:
conn.execute('ALTER TABLE results ADD COLUMN date_ts DATE')
conn.execute('''
UPDATE results
SET date_ts = DATE(date)
''')
conn.commit()

In [13]:
conn.execute('ALTER TABLE results ADD COLUMN mindate_ts DATE')
conn.execute('ALTER TABLE results ADD COLUMN maxdate_ts DATE')
conn.execute('''
UPDATE results
SET mindate_ts = DATE(mindate),
    maxdate_ts = DATE(maxdate)
''')
conn.commit()

In [16]:
conn.execute('ALTER TABLE results ADD COLUMN valid INTEGER')
conn.execute('''
UPDATE results
SET valid = CASE
    WHEN date_ts BETWEEN mindate_ts AND maxdate_ts THEN 1
    ELSE 0
END
''')
conn.commit()

In [18]:
pd.read_sql('PRAGMA table_info(results)', conn)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,date,TEXT,0,None,0
2,2,home_team,TEXT,0,None,0
3,3,away_team,TEXT,0,None,0
4,4,home_score,INTEGER,0,None,0
5,5,away_score,INTEGER,0,None,0
6,6,tournament,TEXT,0,None,0
7,7,city,TEXT,0,None,0
8,8,country,TEXT,0,None,0
9,9,neutral,INTEGER,0,None,0


In [6]:
pd.read_sql('''
SELECT *
FROM results
limit 3
''', conn)

,index,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,noaa_country_code,location_id,mindate,maxdate,date_ts,mindate_ts,maxdate_ts,valid,location_date_id
0,0,1872-11-30 00:00:00,Scotland,England,0,0,Friendly,Glasgow,United Kingdom,0,UK,CITY:UK000006,1914-01-01,2024-08-31,1872-11-30,1914-01-01,2024-08-31,0,CITY:UK000006_1872-11-30
1,1,1873-03-08 00:00:00,England,Scotland,4,2,Friendly,London,United Kingdom,0,UK,CITY:UK000009,1904-02-01,2024-09-24,1873-03-08,1904-02-01,2024-09-24,0,CITY:UK000009_1873-03-08
2,2,1874-03-07 00:00:00,Scotland,England,2,1,Friendly,Glasgow,United Kingdom,0,UK,CITY:UK000006,1914-01-01,2024-08-31,1874-03-07,1914-01-01,2024-08-31,0,CITY:UK000006_1874-03-07


In [19]:
pd.read_sql('''
select
count(*) as total,
sum(case when location_id is null then 1 end) as location_id_nulls,
sum(case when location_id is not null then 1 end) as location_id_not_nulls,
sum(case when valid = 0 then 1 end) as invalid_dates,
sum(case when valid = 1 then 1 end) as valid_dates,
sum(valid = 1 and location_id is not null) as valid_and_location_id_not_nulls
from results
''', conn) 

,total,location_id_nulls,location_id_not_nulls,invalid_dates,valid_dates,valid_and_location_id_not_nulls
0,47598,16221,31377,19127,28471,28471


In [38]:
pd.read_sql('''
select *
from locations
limit 3
''', conn)

,id,name,mindate,maxdate,datacoverage,city,noaa_country_code
0,CITY:AE000001,"Abu Dhabi, AE",1983-01-01,2024-09-24,0.9974,Abu Dhabi,AE
1,CITY:AE000002,"Ajman, AE",1944-03-01,2024-09-24,1.0000,Ajman,AE
2,CITY:AE000003,"Dubai, AE",1944-03-01,2024-09-24,1.0000,Dubai,AE


### add unique id for each location_id + date_ts

In [5]:
conn.execute('ALTER TABLE results ADD COLUMN location_date_id TEXT')
conn.execute('''
UPDATE results
SET location_date_id = location_id || '_' || date_ts
''')
conn.commit()